In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
amphet = pd.read_csv('../data/drugs/US_Amphetamine.csv')

In [3]:
amphet.head()

,placeDcid,placeName,Date:RetailDrugDistribution_DrugDistribution_Amphetamine,Value:RetailDrugDistribution_DrugDistribution_Amphetamine,Source:RetailDrugDistribution_DrugDistribution_Amphetamine
0,geoId/01001,Autauga County,2006-01,301.62,https://www.deadiversion.usdoj.gov/arcos/retai...
1,geoId/01001,Autauga County,2006-04,277.00,https://www.deadiversion.usdoj.gov/arcos/retai...
2,geoId/01001,Autauga County,2006-07,280.13,https://www.deadiversion.usdoj.gov/arcos/retai...
3,geoId/01001,Autauga County,2006-10,323.46,https://www.deadiversion.usdoj.gov/arcos/retai...
4,geoId/01001,Autauga County,2007-01,341.98,https://www.deadiversion.usdoj.gov/arcos/retai...


In [ ]:
# Use detected encoding
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

class CountyNormalizer:
    def __init__(self):
        self.data2000_09 = CountyNormalizer.clean(pd.read_csv('../data/population/co-est2009-alldata.csv', dtype={c: str for c in ['COUNTY', 'STATE', 'SUMLEV']},  encoding='ISO-8859-1'))
        self.data2010_19 = CountyNormalizer.clean(pd.read_csv('../data/population/co-est2020-alldata.csv', dtype={c: str for c in ['COUNTY', 'STATE', 'SUMLEV']}, encoding='ISO-8859-1'))
        self.data2020_23 = CountyNormalizer.clean(pd.read_csv('../data/population/co-est2023-alldata.csv', dtype={c: str for c in ['COUNTY', 'STATE', 'SUMLEV']}, encoding='ISO-8859-1'))
        self.id_to_row_00_09 = CountyNormalizer.id_to_row(self.data2000_09)
        self.id_to_row_10_19 = CountyNormalizer.id_to_row(self.data2010_19)
        self.id_to_row_20_23 = CountyNormalizer.id_to_row(self.data2020_23)
        
        self.cache = {}

    def clean(df):
        # remove all rows where CTYNAME == STNAME, except one instance of district of columbia; we arbitrarily keep the COUNTY == 0
        df = df[(df['STNAME'] != df['CTYNAME']) | ((df['STNAME'] == 'District of Columbia') & (df['COUNTY'] == 0))].reset_index()
        return df
    
    def id_to_row(df):
        def concat_state_and_county(row):
            return row['STATE'] + row['COUNTY']
        county_ids = df.apply(concat_state_and_county, axis=1)
        id_to_row_map = {}
        for i in range(len(county_ids)):
            id_to_row_map[county_ids.iloc[i]] = i
        return id_to_row_map
    
    def clear_cache(self):
        del self.cache
        self.cache = {}
        
    def get_population(self, countyId, year):
        '''
        countyName: string
        year: int
        '''
        assert isinstance(year, int) and isinstance(countyId, str)
        assert 2000 <= year <= 2023
        if f'{countyId}{year}' in self.cache:
            return self.cache[f'{countyId}{year}']
        
        data = None
        id_to_row = None
        
        if year < 2010:
            data = self.data2000_09
            id_to_row = self.id_to_row_00_09
        elif year < 2020:
            data = self.data2010_19
            id_to_row = self.id_to_row_10_19
        else:
            data = self.data2020_23
            id_to_row = self.id_to_row_20_23
        
        self.cache[f'{countyId}{year}'] = data[f'POPESTIMATE{year}'].iloc[id_to_row[countyId]]
        return self.cache[f'{countyId}{year}']
        


In [39]:
c = CountyNormalizer()


In [45]:
c.get_population('56045', 2021)

6746

In [44]:
c.data2010_19

,index,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,POPESTIMATE2020,NPOPCHG_2010,NPOPCHG_2011,NPOPCHG_2012,NPOPCHG_2013,NPOPCHG_2014,NPOPCHG_2015,NPOPCHG_2016,NPOPCHG_2017,NPOPCHG_2018,NPOPCHG_2019,NPOPCHG_2020,BIRTHS2010,BIRTHS2011,BIRTHS2012,BIRTHS2013,BIRTHS2014,BIRTHS2015,BIRTHS2016,BIRTHS2017,BIRTHS2018,BIRTHS2019,BIRTHS2020,DEATHS2010,DEATHS2011,DEATHS2012,DEATHS2013,DEATHS2014,DEATHS2015,DEATHS2016,DEATHS2017,DEATHS2018,DEATHS2019,DEATHS2020,NATURALINC2010,NATURALINC2011,NATURALINC2012,NATURALINC2013,NATURALINC2014,NATURALINC2015,NATURALINC2016,NATURALINC2017,NATURALINC2018,NATURALINC2019,NATURALINC2020,INTERNATIONALMIG2010,INTERNATIONALMIG2011,INTERNATIONALMIG2012,INTERNATIONALMIG2013,INTERNATIONALMIG2014,INTERNATIONALMIG2015,INTERNATIONALMIG2016,INTERNATIONALMIG2017,INTERNATIONALMIG2018,INTERNATIONALMIG2019,INTERNATIONALMIG2020,DOMESTICMIG2010,DOMESTICMIG2011,DOMESTICMIG2012,DOMESTICMIG2013,DOMESTICMIG2014,DOMESTICMIG2015,DOMESTICMIG2016,DOMESTICMIG2017,DOMESTICMIG2018,DOMESTICMIG2019,DOMESTICMIG2020,NETMIG2010,NETMIG2011,NETMIG2012,NETMIG2013,NETMIG2014,NETMIG2015,NETMIG2016,NETMIG2017,NETMIG2018,NETMIG2019,NETMIG2020,RESIDUAL2010,RESIDUAL2011,RESIDUAL2012,RESIDUAL2013,RESIDUAL2014,RESIDUAL2015,RESIDUAL2016,RESIDUAL2017,RESIDUAL2018,RESIDUAL2019,RESIDUAL2020,GQESTIMATESBASE2010,GQESTIMATES2010,GQESTIMATES2011,GQESTIMATES2012,GQESTIMATES2013,GQESTIMATES2014,GQESTIMATES2015,GQESTIMATES2016,GQESTIMATES2017,GQESTIMATES2018,GQESTIMATES2019,GQESTIMATES2020,RBIRTH2011,RBIRTH2012,RBIRTH2013,RBIRTH2014,RBIRTH2015,RBIRTH2016,RBIRTH2017,RBIRTH2018,RBIRTH2019,RBIRTH2020,RDEATH2011,RDEATH2012,RDEATH2013,RDEATH2014,RDEATH2015,RDEATH2016,RDEATH2017,RDEATH2018,RDEATH2019,RDEATH2020,RNATURALINC2011,RNATURALINC2012,RNATURALINC2013,RNATURALINC2014,RNATURALINC2015,RNATURALINC2016,RNATURALINC2017,RNATURALINC2018,RNATURALINC2019,RNATURALINC2020,RINTERNATIONALMIG2011,RINTERNATIONALMIG2012,RINTERNATIONALMIG2013,RINTERNATIONALMIG2014,RINTERNATIONALMIG2015,RINTERNATIONALMIG2016,RINTERNATIONALMIG2017,RINTERNATIONALMIG2018,RINTERNATIONALMIG2019,RINTERNATIONALMIG2020,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RDOMESTICMIG2016,RDOMESTICMIG2017,RDOMESTICMIG2018,RDOMESTICMIG2019,RDOMESTICMIG2020,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019,RNETMIG2020
0,1,050,3,6,01,001,Alabama,Autauga County,54571,54582,54761,55229,54970,54747,54922,54903,55302,55448,55533,55769,56145,179,468,-259,-223,175,-19,399,146,85,236,376,151,639,615,570,637,652,675,667,651,592,606,157,514,560,584,572,585,547,574,563,552,582,-6,125,55,-14,65,67,128,93,88,40,24,28,16,0,19,19,23,7,-3,2,-14,-8,147,327,-329,-226,102,-107,266,58,-3,208,360,175,343,-329,-207,121,-84,273,55,-1,194,352,10,0,15,-2,-11,-2,-2,-2,-2,2,0,455,455,455,455,455,455,455,455,455,455,455,455,11.619238,11.161626,10.390368,11.616774,11.873435,12.249898,12.045147,11.731738,10.637724,10.829744,9.346304,10.163432,10.645570,10.431389,10.653312,9.926954,10.365688,10.145881,9.918959,10.400844,2.272934,0.998194,-0.255202,1.185385,1.220123,2.322944,1.679458,1.585857,0.718765,0.428901,0.290936,0.000000,0.346346,0.346497,0.418848,0.127036,-0.054176,0.036042,-0.251568,-0.142967,5.945995,-5.971016,-4.119690,1.860143,-1.948555,4.827367,1.047404,-0.054063,3.737579,6.433511,6.236931,-5.971016,-3.773344,2.206640,-1.529706,4.954403,0.993228,-0.018021,3.486011,6.290545
1,2,050,3,6,01,003,Alabama,Baldwin County,182265,182263,183121,186579,190203,194978,199306,203101,207787,212737,218071,223565,229287,858,3458,3624,4775,4328,3795,4686,4950,5334,5494,5722,514,2186,2092,2162,2215,2261,2286,2313,2297,2322,2317,534,1829,1883,1902,1988,2099,2021,2103,2325,2386,2543,-20,357,209,260,227,162,265,210,-28,-64,-226,51,192,273,240,141,165,211,104

In [22]:
counties = set()
for county in c.data2000_09['CTYNAME']:
    if county in counties:
        print(f'repeat of {county}')
    counties.add(county)

repeat of Calhoun County
repeat of Clay County
repeat of Cleburne County
repeat of Dallas County
repeat of Franklin County
repeat of Greene County
repeat of Jackson County
repeat of Jefferson County
repeat of Lawrence County
repeat of Lee County
repeat of Madison County
repeat of Marion County
repeat of Monroe County
repeat of Montgomery County
repeat of Perry County
repeat of Pike County
repeat of Randolph County
repeat of Washington County
repeat of Nevada County
repeat of Santa Cruz County
repeat of Jackson County
repeat of Jefferson County
repeat of Lake County
repeat of Lincoln County
repeat of Logan County
repeat of Morgan County
repeat of Phillips County
repeat of Washington County
repeat of Yuma County
repeat of Calhoun County
repeat of Clay County
repeat of Columbia County
repeat of Escambia County
repeat of Franklin County
repeat of Jackson County
repeat of Jefferson County
repeat of Lafayette County
repeat of Lake County
repeat of Lee County
repeat of Madison County
repeat o